In [1]:
import pandas as pd
import geopandas as gp
import numpy as np 
import os
import fiona
from statistics import mean, median
import string
import sys

# Oregon 2018

## VEST Documentation

Election results from the Oregon Secretary of State via OpenElections (https://github.com/openelections/openelections-data-or/).

Precinct shapefiles for the following counties were provided by the respective county governments: Benton, Clackamas, Clatsop, Columbia, Coos, Crook, Curry, Deschutes, Douglas, Harney, Hood River, Jackson, Josephine, Klamath, Lane, Linn, Malheur, Morrow, Multnomah, Polk, Sherman, Tillamook, Umatilla, Union, Wallowa, Wasco, Washington, Yamhill.

Precinct shapefiles for the following counties were provided by the Oregon Secretary of State: Jefferson, Lane, Marion, Washington. Several gaps in the Marion County shapefile were assigned based on the Precinct Split Summary Report.

Precinct boundaries for Grant County and Wheeler County are defined by school district boundaries. Precinct shapefiles were produced to match PDF maps provided by the respective counties using the 2016 Oregon Education Boundaries shapefile obtained from the Oregon Spatial Data Library.

Precinct shapefiles for the following counties were produced based on PDF maps provided by the respective counties: Baker, Gilliam, Lake, Lincoln. Boundaries defined by the PLSS grid were produced with the Oregon Public Land Survey Quarter-Quarter Reference Grid shapefile obtained from the Oregon Spatial Data Library. Municipal boundaries were produced from the Oregon Department of Transportion 2016 City Limits shapefile obtained from the Oregon Spatial Data Library. Street boundaries were produced from the U.S. Census Bureau census block shapefiles.

PDF maps obtained from Lake County are approximate precinct boundaries drawn on the PLSS grid for county precincts and on the street map for Lakeview city precincts. These boundaries were adjusted to match address range assignments in the Lake County Address Library Report which serves as the legal definition of the precincts according to the Lake County Clerk. Precinct divisions across roads that span multiple rural precincts are generally defined by zip codes. Address ranges were further identified based on the Lake County tax lot parcel viewer.

Rural precinct boundaries in the shapefiles provided by Coos County and Union County were revised to match the updated PLSS cadastral grid in the Oregon GIS Framework obtained from the Oregon Spatial Data Library. 

Municipal precinct boundaries in the following counties were edited to match city limits in effect for the November 2018 general election: Clackamas, Coos, Deschutes, Douglas, Marion, Tillamook, Umatilla, Washington, Yamhill.

Precinct numbers in the Columbia County, Josephine County, and Wallowa County shapefiles were edited to match the Oregon Secretary of State voter file.

G18GOVDBRO - Kate Brown (Democratic Party)  
G18GOVRBUE - Knute Buehler (Republican Party)  
G18GOVISTA - Patrick Starnes (Independent Party)  
G18GOVLCHE - Nick Chen (Libertarian Party)  
G18GOVCAUE - Aaron Auer (Constitution Party)  
G18GOVPHEN - Chris Henry (Progressive Party)  
G18GOVOWRI - Write-in Votes  

## Load VEST File

In [2]:
vest_or_18 = gp.read_file("./raw-from-source/VEST/or_2018/or_2018.shp")

In [3]:
data_columns = [i for i in vest_or_18.columns if "G18" in i]

## Load OR Votes

In [4]:
file_list = os.listdir("./raw-from-source/Election_Results/2018/counties")
li = []
for file in file_list:
    #print(file)
    if "general" in file:
        #print(file)
        file_ref = "./raw-from-source/Election_Results/2018/counties/"+file
        file_prev = pd.read_csv(file_ref)
        #if 'precincts' in file_prev.columns:
            #file_prev['precinct'] = file_prev['precincts']
     
        #file_prev = file_prev[['county', 'precinct', 'office', 'district', 'party', 'candidate','votes']]
        li.append(file_prev)
election_results = pd.concat(li, axis=0, ignore_index=True)
display(election_results)

,county,precinct,office,district,party,candidate,votes
0,Yamhill,"Precinct 001 - Newberg Rural N, S & E",Governor,NaN,NaN,Write-ins,2
1,Yamhill,"Precinct 001 - Newberg Rural N, S & E",Governor,NaN,CON,Aaron Auer,22
2,Yamhill,"Precinct 001 - Newberg Rural N, S & E",Governor,NaN,DEM,Kate Brown,708
3,Yamhill,"Precinct 001 - Newberg Rural N, S & E",Governor,NaN,LBT,Nick Chen,23
4,Yamhill,"Precinct 001 - Newberg Rural N, S & E",Governor,NaN,NON,Patrick Starnes,44
...,...,...,...,...,...,...,...
23753,Wheeler,SPR,State Senate,30.0,REP,Cliff Bentz,201
23754,Wheeler,SPR,U.S. House,2.0,NaN,Write-ins,0
23755,Wheeler,SPR,U.S. House,2.0,DEM,Jamie McLeod-Skinner,48
23756,Wheeler,SPR,U.S. House,2.0,NON,Mark R Roberts,14


In [5]:
election_results["office"].unique()

array(['Governor', 'State House', 'State Senate', 'U.S. House', nan,
       'Ballots Cast - Blank', '20-120 Ridgewood Road District',
       'Registered Voters', 'Ballots Cast'], dtype=object)

In [6]:
election_results[election_results["office"].isna()]

,county,precinct,office,district,party,candidate,votes
4901,Benton,1,NaN,NaN,NaN,Registered Voters,4179
4902,Benton,1,NaN,NaN,NaN,Total Votes Cast,3381
4930,Benton,2,NaN,NaN,NaN,Registered Voters,3162
4931,Benton,2,NaN,NaN,NaN,Total Votes Cast,2131
4959,Benton,3,NaN,NaN,NaN,Registered Voters,4079
4960,Benton,3,NaN,NaN,NaN,Total Votes Cast,3125
4988,Benton,4,NaN,NaN,NaN,Registered Voters,2037
4989,Benton,4,NaN,NaN,NaN,Total Votes Cast,1426
5017,Benton,5,NaN,NaN,NaN,Registered Voters,2998
5018,Benton,5,NaN,NaN,NaN,Total Votes Cast,2082


In [7]:
election_results = election_results[election_results["office"]=="Governor"]

In [8]:
election_results

,county,precinct,office,district,party,candidate,votes
0,Yamhill,"Precinct 001 - Newberg Rural N, S & E",Governor,NaN,NaN,Write-ins,2
1,Yamhill,"Precinct 001 - Newberg Rural N, S & E",Governor,NaN,CON,Aaron Auer,22
2,Yamhill,"Precinct 001 - Newberg Rural N, S & E",Governor,NaN,DEM,Kate Brown,708
3,Yamhill,"Precinct 001 - Newberg Rural N, S & E",Governor,NaN,LBT,Nick Chen,23
4,Yamhill,"Precinct 001 - Newberg Rural N, S & E",Governor,NaN,NON,Patrick Starnes,44
...,...,...,...,...,...,...,...
23743,Wheeler,SPR,Governor,NaN,DEM,Kate Brown,39
23744,Wheeler,SPR,Governor,NaN,LBT,Nick Chen,5
23745,Wheeler,SPR,Governor,NaN,NON,Patrick Starnes,12
23746,Wheeler,SPR,Governor,NaN,PRO,Chris Henry,1


In [9]:
election_results["pivot_col"] = election_results["county"] +"-" + election_results["precinct"].astype(str)

In [10]:
election_results["candidate"].unique()

array(['Write-ins', 'Aaron Auer', 'Kate Brown', 'Nick Chen',
       'Patrick Starnes', 'Chris Henry', 'Knute Buehler', 'write-in',
       'Over Votes', 'Under Votes', 'Write-In', 'Write-Ins',
       'Write-In Totals', 'Overvotes', 'Undervotes', 'write-ins',
       'Write-in'], dtype=object)

In [11]:
cand_rename_dict = {'Write-ins':"G18GOVOWRI", 
 'Aaron Auer':"G18GOVCAUE", 
 'Kate Brown':"G18GOVDBRO", 
 'Nick Chen':"G18GOVLCHE",
 'Patrick Starnes':"G18GOVISTA", 
 'Chris Henry':"G18GOVPHEN", 
 'Knute Buehler':"G18GOVRBUE", 
 'write-in':"G18GOVOWRI",
 'Over Votes':"drop",
 'Under Votes':"drop", 
 'Write-In':"G18GOVOWRI",
 'Write-Ins':"G18GOVOWRI",
 'Write-In Totals':"G18GOVOWRI",
 'Overvotes':"drop", 
 'Undervotes':"drop", 
 'write-ins':"G18GOVOWRI",
 'Write-in':"G18GOVOWRI"}

In [12]:
election_results["candidate"] = election_results["candidate"].map(cand_rename_dict).fillna(election_results["candidate"])

In [13]:
election_results = election_results[election_results["candidate"]!="drop"]

In [14]:
pivoted_results = pd.pivot_table(election_results,index="pivot_col",values=["votes"],aggfunc=sum,columns="candidate")

In [15]:
pivoted_results.reset_index(inplace=True,drop=False)

In [16]:
pivoted_results.columns = pivoted_results.columns.droplevel(0)

In [17]:
pivoted_results.rename(columns={"":"unique_ID"},inplace=True)

In [18]:
pivoted_results.head()

candidate,unique_ID,G18GOVCAUE,G18GOVDBRO,G18GOVISTA,G18GOVLCHE,G18GOVOWRI,G18GOVPHEN,G18GOVRBUE
0,Baker-Baker #1,10,186,33,16,1,9,549
1,Baker-Baker #2,10,175,42,18,2,7,520
2,Baker-Baker #3,15,186,38,11,1,12,587
3,Baker-Baker #4,10,186,46,16,3,8,758
4,Baker-Baker #5,25,189,50,27,2,3,776


## Statewide Check

In [19]:
def statewide_totals_check(partner_df,source_df,column_list):
    """Compares the totals of two election result dataframes at the statewide total level

    Args:
      partner_df: DataFrame of election results we are comparing against
      source_df: DataFrame of election results we are comparing to
      column_list: List of races that there are votes for
 
    Returns:
      Nothing, only prints out an analysis
    """
    print("***Statewide Totals Check***")
    for race in column_list:
        if (partner_df[race].sum()- source_df[race].sum() != 0):
            print(race+" has a difference of "+str(partner_df[race].sum()-source_df[race].sum())+" votes")
            print("\tVEST: "+str(partner_df[race].sum())+" votes")
            print("\tSOURCES: "+str(source_df[race].sum())+" votes")
        else:
            print(race + " is equal", "\tVEST / RDH: " + str(partner_df[race].sum()))

In [20]:
pivoted_results.fillna(0)

for i in data_columns:
    pivoted_results[i] = pivoted_results[i].astype(str)
    pivoted_results[i] = pivoted_results[i].str.replace(",","")
    pivoted_results[i] = pivoted_results[i].str.replace("nan","0")
    pivoted_results[i] = pivoted_results[i].astype(int)

In [21]:
pivoted_results = pivoted_results[~pivoted_results["unique_ID"].isin(["Clackamas-CLACKAMAS OR","Coos-COOS OR",'Crook-CROOK OR', 'Grant-GRANT OR', 'Jackson-Total', 'Josephine-JOSEPHINE OR', 'Lane-LANE OR', 'Linn-LINN OR', 'Tillamook-Total', 'Union-UNION OR'])]

In [22]:
pivoted_results.head(30)

candidate,unique_ID,G18GOVCAUE,G18GOVDBRO,G18GOVISTA,G18GOVLCHE,G18GOVOWRI,G18GOVPHEN,G18GOVRBUE
0,Baker-Baker #1,10,186,33,16,1,9,549
1,Baker-Baker #2,10,175,42,18,2,7,520
2,Baker-Baker #3,15,186,38,11,1,12,587
3,Baker-Baker #4,10,186,46,16,3,8,758
4,Baker-Baker #5,25,189,50,27,2,3,776
5,Baker-Baker Country,11,83,19,3,0,0,574
6,Baker-Durkee,2,6,1,0,0,0,66
7,Baker-Eagle VA,6,60,12,9,1,0,284
8,Baker-Haines,9,84,17,6,1,2,420
9,Baker-Hereford,1,4,1,0,0,1,64


In [23]:
statewide_totals_check(vest_or_18,pivoted_results,data_columns)

***Statewide Totals Check***
G18GOVDBRO is equal 	VEST / RDH: 934498
G18GOVRBUE is equal 	VEST / RDH: 814988
G18GOVISTA is equal 	VEST / RDH: 53392
G18GOVLCHE is equal 	VEST / RDH: 28927
G18GOVCAUE is equal 	VEST / RDH: 21145
G18GOVPHEN is equal 	VEST / RDH: 11013
G18GOVOWRI is equal 	VEST / RDH: 3034


In [24]:
def county_totals_check(partner_df,source_df,column_list,county_col,full_print=False):
    """Compares the totals of two election result dataframes at the county level

    Args:
      partner_df: DataFrame of election results we are comparing against
      source_df: DataFrame of election results we are comparing to
      column_list: List of races that there are votes for
      county_col: String of the column name that contains county information
      full_print: Boolean specifying whether to print out everything, including counties w/ similarities

    Returns:
      Nothing, only prints out an analysis
    """
    
    print("***Countywide Totals Check***")
    print("")
    diff_counties=[]
    for race in column_list:
        diff = partner_df.groupby([county_col]).sum()[race]-source_df.groupby([county_col]).sum()[race]
        for val in diff[diff != 0].index.values.tolist():
            if val not in diff_counties:
                diff_counties.append(val)
        if len(diff[diff != 0]!=0):   
            print(race + " contains differences in these counties:")
            for val in diff[diff != 0].index.values.tolist():
                county_differences = diff[diff != 0]
                print("\t"+val+" has a difference of "+str(county_differences[val])+" votes")
                print("\t\tVEST: "+str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
                print("\t\tSOURCES: "+str(source_df.groupby([county_col]).sum().loc[val,race])+" votes")
            if (full_print):
                for val in diff[diff == 0].index.values.tolist():
                    county_similarities = diff[diff == 0]
                    print("\t"+val + ": "+ str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
        else:
            print(race + " is equal across all counties")
            if (full_print):
                for val in diff[diff == 0].index.values.tolist():
                    county_similarities = diff[diff == 0]
                    print("\t"+val + ": "+ str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
    if (len(diff_counties)>0):
        print()
        print(diff_counties)

In [25]:
pivoted_results["COUNTY"] = pivoted_results["unique_ID"].apply(lambda x:x.split("-")[0])

In [26]:
county_totals_check(vest_or_18,pivoted_results,data_columns,"COUNTY",full_print=False)

***Countywide Totals Check***

G18GOVDBRO is equal across all counties
G18GOVRBUE is equal across all counties
G18GOVISTA is equal across all counties
G18GOVLCHE is equal across all counties
G18GOVCAUE is equal across all counties
G18GOVPHEN is equal across all counties
G18GOVOWRI is equal across all counties


## Precinct-by-Precinct Check

In [28]:
vest_or_18["unique_ID"]=vest_or_18["COUNTY"]+"-"+vest_or_18["NAME"]+" "+vest_or_18["PRECINCT"]
vest_or_18["unique_ID"] = vest_or_18["unique_ID"].str.upper()

In [30]:
pivoted_results["unique_ID"] = pivoted_results["unique_ID"].str.upper()

In [38]:
vest_or_18["unique_ID"].sort_values()

340                         Baker-Baker 13
337                  Baker-Baker City 1 01
338                  Baker-Baker City 2 02
341                  Baker-Baker City 3 03
339                  Baker-Baker City 4 04
                       ...                
998         Yamhill-Newberg Rural N,S,E 01
999             Yamhill-Newberg Rural W 09
26        Yamhill-Sheridan City & Rural 26
689     Yamhill-Willaminia City & Rural 27
1338       Yamhill-Yamhill City & Rural 13
Name: unique_ID, Length: 1340, dtype: object

In [39]:
pivoted_results["unique_ID"].sort_values()

0                                       BAKER-BAKER #1
1                                       BAKER-BAKER #2
2                                       BAKER-BAKER #3
3                                       BAKER-BAKER #4
4                                       BAKER-BAKER #5
                             ...                      
1340       YAMHILL-PRECINCT 021 - CARLTON CITY & RURAL
1341     YAMHILL-PRECINCT 022 - LAFAYETTE CITY & RURAL
1342      YAMHILL-PRECINCT 026 - SHERIDAN CITY & RURAL
1343    YAMHILL-PRECINCT 027 - WILLAMINIA CITY & RURAL
1344           YAMHILL-PRECINCT 028 - GRAND RONDE AREA
Name: unique_ID, Length: 1335, dtype: object

In [33]:
join_attempt_one = pd.merge(vest_or_18,pivoted_results,on="unique_ID",how="outer",indicator=True)

In [40]:
join_attempt_one["_merge"].value_counts()

join_attempt_one[join_attempt_one["_merge"]=="left_only"].to_csv("./vest_only.csv")
join_attempt_one[join_attempt_one["_merge"]=="right_only"].to_csv("./pivoted.csv")

In [41]:
source_vest_id_changes = pd.read_csv("./source_vest_id_changes.csv")
source_vest_id_changes_dict = dict(zip(source_vest_id_changes["election_ID"],source_vest_id_changes["vest_ID"]))

In [42]:
pivoted_results["unique_ID"] = pivoted_results["unique_ID"].map(source_vest_id_changes_dict).fillna(pivoted_results["unique_ID"])

In [43]:
join_attempt_two = pd.merge(vest_or_18,pivoted_results,on="unique_ID",how="outer",indicator=True)
join_attempt_two["_merge"].value_counts()

#join_attempt_two[join_attempt_two["_merge"]=="left_only"].to_csv("./vest_only.csv")
#join_attempt_two[join_attempt_two["_merge"]=="right_only"].to_csv("./pivoted.csv")

both          1335
left_only        5
right_only       0
Name: _merge, dtype: int64

## Precinct-by-Precinct

In [45]:
def precinct_votes_check(merged_df,column_list,vest_on_left,name_col,print_level=0):
    """Checks a merged dataframe with two election results at the precinct level

    Args:
      merged_df: DataFrame with one set of election results joined to another
      column_list: List of races that there are votes for
      vest_on_left: Boolean specifying whether VEST data is on the left side of merged_df
      name_col: String of the column name to refer to precincts when a difference occurs
      print_level: Integer that specifies how large the vote difference in a precinct must be to be printed

    Returns:
      Nothing, only prints out an analysis
    """
    merged_df = merged_df.sort_values(by=[name_col],inplace=False)
    matching_rows = 0
    different_rows = 0
    diff_list=[]
    diff_values = []
    max_diff = 0
    for index,row in merged_df.iterrows():
        same = True
        for i in column_list:
            left_data = i + "_x"
            right_data = i + "_y"
            if ((row[left_data] is None) or (row[right_data] is None) or (np.isnan(row[right_data])or(np.isnan(row[left_data])))):
                print("FIX NaN value at: ", row[name_col])
                return;
            diff = abs(row[left_data]-row[right_data])
            if (diff>0):
                same = False
                diff_values.append(abs(diff))
                if (diff>max_diff):
                    max_diff = diff
            if(diff>print_level):
                if (vest_on_left):
                    print(i, "{:.>72}".format(row[name_col]), "(V)","{:.>5}".format(int(row[left_data]))," (S){:.>5}".format(int(row[right_data])),"(D):{:>5}".format(int(row[left_data]-row[right_data])))                           
                else:
                    print(i, "{:.>72}".format(row[name_col]), "(S)","{:.>5}".format(int(row[left_data]))," (V){:.>5}".format(int(row[right_data])),"(D):{:>5}".format(int(row[left_data]-row[right_data])))
        if(same != True):
            different_rows +=1
            diff_list.append(row[name_col])
        else:
            matching_rows +=1
    print("")
    print("There are ", len(merged_df.index)," total rows")
    print(different_rows," of these rows have election result differences")
    print(matching_rows," of these rows are the same")
    print("")
    print("The max difference between any one shared column in a row is: ", max_diff)
    if(len(diff_values)!=0):
        print("The average difference is: ", str(sum(diff_values)/len(diff_values)))
    count_big_diff = len([i for i in diff_values if i > 10])
    print("There are ", str(count_big_diff), "precinct results with a difference greater than 10")
    print("")
    print("All precincts containing differences:")
    diff_list.sort()
    print(diff_list)

In [47]:
precinct_votes_check(join_attempt_two[join_attempt_two["_merge"]=="both"],data_columns,True,"unique_ID",print_level=0)


There are  1335  total rows
0  of these rows have election result differences
1335  of these rows are the same

The max difference between any one shared column in a row is:  0
There are  0 precinct results with a difference greater than 10

All precincts containing differences:
[]
